# 0. data load & table info

### plotly

- 온라인 데이터 분석 및 시각화 툴
- 그래프 호스팅을 위한 웹 서비스를 제공 ; 무료 acount 생성이 필요.
- 그래프는 온라인 plotly 계정에 저장, 개인정보를 설정할 수 있음.
- 아래에서 username 에 본인의 계정이름 / api_key에 본인의 API Key를 전달해야함.
- https://chart-studio.plotly.com/feed/#/
<br>

#### 플로팅 온라인 시작하기
- 온라인 플로팅에는 두 가지 방법이 있음.
> 1. py.plot() ; 고유한 URL을 반환, 선택적으로 url을 open
> 2. py.iplot() ; 주피터 노트북에 플로을 표시함.

In [23]:
# 패키지 불러오기
# 주요 시각화 패키지 : matplotlib, seaborn, plotly
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
# plotly.plotly x -> version 4부터 많은 부분이 chart_studio로 이전
# chart_studio.plotly로 호출
from chart_studio.plotly import iplot
import warnings
# chart_studio 별도로 불러오기
import chart_studio

warnings.filterwarnings("ignore")
# plotly.tools에 set_credentials_file() x
# 이 역시 version 4 부터 chart_studio로 이전
# plotly.tools.set_credentials_file(username='ID', api_key='APIKEY')
chart_studio.tools.set_credentials_file(username='fufurin', api_key="choi0415")
data_path = "../../kaggle_data/credit_card_visualization/data/train.csv"

In [24]:
%matplotlib inline
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [25]:
train = pd.read_csv(data_path)

In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3362796 entries, 0 to 3362795
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   store_id      int64  
 1   date          object 
 2   time          object 
 3   card_id       object 
 4   amount        int64  
 5   installments  float64
 6   days_of_week  int64  
 7   holyday       int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 205.2+ MB


### Field Name & Field Details

- store_id : 각 파일에서의 상점 고유 번호.
- data : 거래 일자
- time : 거래 시간
- card_id : 카드 번호의 hash 값
- amount : 매출액, 0보다 작은 음수는 거래 취소(환불)
- installments : 할부개월수. 일시불은 빈 문자열
- days_of_week : 요일, 월요일이 0, 일용일은 6
- holyday : 1이면 공휴일, 0이면 공휴일 아님.

In [27]:
train.head()

,store_id,date,time,card_id,amount,installments,days_of_week,holyday
0,0,2016-12-14,18:05:31,d297bba73f,5,NaN,2,0
1,0,2016-12-14,18:05:54,d297bba73f,-5,NaN,2,0
2,0,2016-12-19,12:42:31,0880849c05,144,NaN,0,0
3,0,2016-12-19,12:48:08,8b4f9e0e95,66,NaN,0,0
4,0,2016-12-19,13:31:08,7ad237eed0,24,NaN,0,0


## data handling for visualization

In [28]:
# 1. Numbers about Stores 시각화를 위해 데이터 가공

# 시간대별 수치를 구하기 위해 'time' 변수에서 시간 추출
train['time_hour'] = pd.DatetimeIndex(train['time']).hour

# 연도별 수치를 구하기 위해 'date'에서 년 추출
train['year'] = pd.DatetimeIndex(train['date']).year

# 날짜별로 aggregation(집합)
train_agg = pd.DataFrame(train.groupby('date')['amount'].sum())

# 시간별로 aggregation
train_agg2 = pd.DataFrame(train.groupby('time_hour')['amount'].sum())

# 요일별로 aggregation
train_agg3 = pd.DataFrame(train.groupby('days_of_week')['amount'].sum())

# 최다 거래횟수 날짜를 구하기 위해 만든 테이블
top_val = pd.DataFrame(train.groupby('date')['amount'].count())

# 카드별(이용자별)로 aggregation
train_user = train.groupby(['card_id']).count()

# 카드별(이용자별) 금액 합계를 구하기 위한 sum aggregation
train_user2 = train.groupby(['card_id']).sum()

# 1. Numbers about Stores

## 1-1. date related graph

In [29]:
## 일자별 거래 횟수

# 데이터 준비하기(for plotly)
x2016 = train.date[train.year == 2016]
x2017 = train.date[train.year == 2017]
x2018 = train.date[train.year == 2018]

# 히스토그램
trace1 = go.Histogram(x = x2016,
                     opacity = 0.75,
                     name = "2016년",
                     marker=dict(color='rgba(171, 50, 96, 0.6)'))

trace2 = go.Histogram(x = x2017,
                     opacity = 0.75,
                     name = "2017년",
                     marker=dict(color='rgba(12, 50, 196, 0.6)'))

trace3 = go.Histogram(x = x2018,
                     opacity = 0.75,
                     name = "2018년",
                     marker=dict(color='rgba(12, 195, 50, 0.6)'))


data = [trace1, trace2, trace3]

# 그래프 레이아웃
layout = go.Layout(barmode='overlay',
                   title='일자별 거래횟수',
                   xaxis=dict(title='날짜'),
                   yaxis=dict( title='거래횟수'))

#Annotation 추가
layout.update(dict(annotations=[go.layout.Annotation(text="최다 거래 건수", x=top_val[top_val.amount == max(top_val.amount) ].index[0], y=max(top_val.amount))]))

#그래프 출력
fig = go.Figure(data=data, layout=layout)
iplot(fig)

PlotlyRequestError: Bad API key